In [1]:
import os
import sys
import math
import locale
import pypdf
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from typing import List
from pydantic import BaseModel, Field
from utils_ccd import get_connection, get_info_file_path, extract_text_from_pdf, generate_pdf_office
from datetime import datetime
from dotenv import load_dotenv
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc
from pathlib import Path
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from docx2pdf import convert

llm = AzureChatOpenAI(model_name="gpt-4o")

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
antecedentes_sql = ''' 
SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

In [3]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(pro.numero_processo, '/', pro.ano_processo) as processo,
        pro.assunto,
        pro.interessado,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.vw_Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    LEFT JOIN processo.dbo.Processos pro
    	ON pro.numero_processo = ai.numero_processo AND pro.ano_processo = ai.ano_processo
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;

'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

In [4]:
def get_transito(nome):
    with open("../consultas/processos_transito_cpf.sql") as f:
        query = f.read()
    q = query.format(nome=nome)
    df = pd.read_sql(q, get_connection())
    df['valor_original'] = df['valor_original'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
    df['valor_atualizado'] = df['valor_atualizado'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
    df.fillna('', inplace=True)
    return df

def get_cpf_responsavel(nome_pessoa, numero_processo, ano_processo):
    query = f'''
SELECT gp.Documento as cpf
	FROM Processo.dbo.Processos p
		INNER JOIN Processo.dbo.Pro_ProcessosResponsavelDespesa pprd 
		ON pprd.IdProcesso = p.IdProcesso
		INNER JOIN Processo.dbo.GenPessoa gp 
		ON gp.IdPessoa = pprd.IdPessoa
	WHERE p.numero_processo = '{numero_processo}'
	AND p.ano_processo = '{ano_processo}'
	AND lower(gp.Nome) = '{nome_pessoa.lower()}'
 '''
    df = pd.read_sql(query, get_connection())

    if len(df) > 1:
        # Corrigido: cpf não existia nesse escopo
        raise ValueError(f"Mais de um CPF para {nome_pessoa}: {df['cpf'].tolist()}")
    elif len(df) == 0:
        return ''
    return df['cpf'].values[0] or ''
    
def get_valores_pessoa(row):
    pessoa_strc = row['pessoas']
    print(f'Pessoas: {pessoa_strc.pessoas}')
    valores = []
    for pessoa in pessoa_strc.pessoas:
        nome_pessoa = pessoa.nome_pessoa
        transitos = get_transito(nome_pessoa)
        cpfs = transitos['cpf'].unique()
        if len(cpfs) > 1:
            print(f'Mais de um cpf por pessoa')
            cpf = ''
        elif len(cpfs) == 0:
            cpf = get_cpf_responsavel(nome_pessoa, row['numero_processo'], row['ano_processo'])
        else:
            cpf = cpfs[0]
        valores.append({'nome': nome_pessoa, 'cpf': cpf, 'transitos': transitos.to_dict(orient='records')})
    return valores

def valores_vazios(valores):
    return all([not bool(v['transitos']) for v in valores])


class PessoaAntecedentes(BaseModel):
  """Pessoa cujo antecedente deve ser buscado"""
  nome_pessoa: str = Field(description="Nome da pessoa para buscar antecedentes")

class PessoasAntecedentes(BaseModel):
  """Lista de pessoas cujos antecedentes devem ser buscados"""
  pessoas: List[PessoaAntecedentes] = Field(description="Lista de pessoas para buscar antecedentes")
  
def get_pessoas_llm(texto_despacho):

  prompt = PromptTemplate.from_template("""
    Você é um agente que identifica listas de pessoas em requisições de antecedentes. 
      Você recebeu uma requisição para buscar antecedentes de pessoas em um processo. Remova qualquer vocativo do nome da pessoa, tal como Sr., Sra., Dr., Dra., etc.
      Não inclua nomes de conselheiros, advogados ou partes do processo. Ex: Sra. Conselheira Substituta Ana
Paula de Oliveira Gomes
      O texto da requisição é o seguinte:
      "{input}"
      Encontre os antecedentes das pessoas listadas na requisição.

    Sua resposta:
    """)

  structured_llm = llm.with_structured_output(schema=PessoasAntecedentes)
  chain = prompt | structured_llm
  return chain.invoke(texto_despacho)
    

In [5]:
df_info_gaana['caminho_arquivo'] = df_info_gaana.apply(get_info_file_path, axis=1)
df_info_gaana['texto_pdf'] = df_info_gaana['caminho_arquivo'].apply(extract_text_from_pdf)
df_info_gaana['pessoas'] = df_info_gaana['texto_pdf'].apply(get_pessoas_llm)

In [6]:
df_info_gaana['valores'] = df_info_gaana.apply(get_valores_pessoa, axis=1)

Pessoas: [PessoaAntecedentes(nome_pessoa='Maria de Fátima Araújo da Silva')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Pedro Lopes de Araújo Neto')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Lawrence Carlos Amorim de Araújo')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Marcos Antonio de Oliveira')]
Mais de um cpf por pessoa
Pessoas: [PessoaAntecedentes(nome_pessoa='Francisco Assis de Medeiros')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Rosania Maria Teixeira Ferreira')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Maria Aparecida Cavalcante')]
Pessoas: [PessoaAntecedentes(nome_pessoa='JOSE NIVALDO ARAUJO DE MELO')]
Pessoas: [PessoaAntecedentes(nome_pessoa='JOÃO PAULO GUEDES LOPES')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Manoel dos Santos Bernardo')]


In [7]:
df_info_gaana

,setor,processo,assunto,interessado,arquivo,datapublicacao,SequencialProcessoEvento,numero_processo,ano_processo,rn,caminho_arquivo,texto_pdf,pessoas,valores
0,GAANA,002407/2024,CONTAS DO CHEFE DO PODER EXECUTIVO DA PREFEITU...,PREFEITURA MUNICIPAL DE OURO BRANCO ...,GAANA_002407_2024_0020.pdf,2025-11-07 09:50:41.840,37,002407,2024,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 2407/...,pessoas=[PessoaAntecedentes(nome_pessoa='Maria...,"[{'nome': 'Maria de Fátima Araújo da Silva', '..."
1,GAANA,002904/2023,REPRESENTAÇÃO ...,CONTROLADORIA GERAL DO ESTADO ...,GAANA_002904_2023_0055.pdf,2025-11-03 10:40:33.427,103,002904,2023,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 2904/...,pessoas=[PessoaAntecedentes(nome_pessoa='Pedro...,"[{'nome': 'Pedro Lopes de Araújo Neto', 'cpf':..."
2,GAANA,003425/2023,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE ALMINO AFONSO ...,GAANA_003425_2023_0029.pdf,2025-11-05 11:01:55.410,54,003425,2023,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 3425/...,pessoas=[PessoaAntecedentes(nome_pessoa='Lawre...,"[{'nome': 'Lawrence Carlos Amorim de Araújo', ..."
3,GAANA,010115/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.PARAZINHO ...,GAANA_010115_2016_0028.pdf,2025-09-29 12:18:11.417,49,010115,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10115...,pessoas=[PessoaAntecedentes(nome_pessoa='Marco...,"[{'nome': 'Marcos Antonio de Oliveira', 'cpf':..."
4,GAANA,010150/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.PARELHAS ...,GAANA_010150_2016_0028.pdf,2025-10-30 13:32:29.970,48,010150,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10150...,pessoas=[PessoaAntecedentes(nome_pessoa='Franc...,"[{'nome': 'Francisco Assis de Medeiros', 'cpf'..."
5,GAANA,010181/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.SERRINHA DOS PINTOS ...,GAANA_010181_2016_0026.pdf,2025-11-05 10:50:25.003,47,010181,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10181...,pessoas=[PessoaAntecedentes(nome_pessoa='Rosan...,"[{'nome': 'Rosania Maria Teixeira Ferreira', '..."
6,GAANA,010248/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.RUI BARBOSA ...,GAANA_010248_2016_0027.pdf,2025-10-17 15:46:41.337,51,010248,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10248...,pessoas=[PessoaAntecedentes(nome_pessoa='Maria...,"[{'nome': 'Maria Aparecida Cavalcante', 'cpf':..."
7,GAANA,010311/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.BAÍA FORMOSA ...,GAANA_010311_2016_0027.pdf,2025-10-24 08:43:41.357,48,010311,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10311...,pessoas=[PessoaAntecedentes(nome_pessoa='JOSE ...,"[{'nome': 'JOSE NIVALDO ARAUJO DE MELO', 'cpf'..."
8,GAANA,010369/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.LAGOA D`ANTA ...,GAANA_010369_2016_0022.pdf,2025-10-30 08:47:40.643,41,010369,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10369...,pessoas=[PessoaAntecedentes(nome_pessoa='JOÃO ...,"[{'nome': 'JOÃO PAULO GUEDES LOPES', 'cpf': '0..."
9,GAANA,200164/2021,INADIMPLÊNCIA NA REMESSA DE CONTAS ANUAIS DE G...,FRANCISCO DE ASSIS PINHEIRO DE ANDRADE ...,GAANA_200164_2021_0051.pdf,2025-10-24 10:16:48.073,96,200164,2021,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_2...,\n \n1 \n \n \nProcesso nº: 20016...,pessoas=[PessoaAntecedentes(nome_pessoa='Manoe...,"[{'nome': 'Manoel dos Santos Bernardo', 'cpf':..."


In [8]:
#df_info_gaana['len_valores'] = df_info_gaana['valores'].apply(lambda x: len(x))
df_info_gaana['sem_valores'] = df_info_gaana['valores'].apply(valores_vazios)
antecedentes = df_info_gaana

In [9]:
for row in antecedentes.itertuples():
    doc = DocxTemplate("templates/antecedentes.docx")
    #valores = [v for v in row.valores if len(v['transitos']) > 0]
    nomes = [v['nome'] for v in row.valores]
    if len(nomes) > 1:
        responsaveis = ', '.join(nomes[:-1]) + ' e ' + nomes[-1]
    else:
        responsaveis = nomes[0] if nomes else ''
    for v in row.valores:
        if v['cpf'] is None:
            v['cpf'] = ''
    context = {
        'processo': row.processo,
        'assunto': row.assunto,
        'interessado': row.interessado,
        'valores': row.valores,
        'responsaveis': responsaveis,
        'data': datetime.now().strftime("%d/%m/%Y"),
        'sem_valores': row.sem_valores
    }
    processo_underline = row.processo.replace('/', '_')

    doc.render(context)
    doc.save(f"saidas/antecedentes/{processo_underline}.docx")
    generate_pdf_office(f"saidas/antecedentes/{processo_underline}.docx", f"saidas/antecedentes/")
    print(f"Gerado arquivo para o processo {row.processo}")

Gerado arquivo para o processo 002407/2024
Gerado arquivo para o processo 002904/2023
Gerado arquivo para o processo 003425/2023
Gerado arquivo para o processo 010115/2016
Gerado arquivo para o processo 010150/2016
Gerado arquivo para o processo 010181/2016
Gerado arquivo para o processo 010248/2016
Gerado arquivo para o processo 010311/2016
Gerado arquivo para o processo 010369/2016
Gerado arquivo para o processo 200164/2021
